## ДЗ_2

### Задание 1.

Обучить три классификатора:
1. На токенах с высокой частотой.
2. На токенах со средней частотой.
3. На токенах с низкой частотой.
Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

### Задание 2.

Найти фичи с наибольшей значимостью, и вывести их.

### Задание 3.

1. Сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report).
2. Подобрать оптимальный размер для hashing векторайзера.
3. Убедиться что для сетки нет переобучения.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# !wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
# !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

Считаем данные и создадим объединенный датасет.

In [3]:
positive = pd.read_csv('./positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('./negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [4]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


Разделим данные на train и test.

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [6]:
# !pip install nltk

In [7]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk import ngrams
from nltk import collocations 
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from collections import Counter

*Отсортируем в тексте "стоп-слова" и пунктуацию.*

In [8]:
nltk.download('genesis')
nltk.download('stopwords')
nltk.download('punkt')
print(stopwords.words('russian'))

[nltk_data] Downloading package genesis to /Users/mac/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data]   Package punkt is already up-to-date!


In [9]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)

*Создадим корпус используемых слов.*

In [10]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2870536


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

*Отсортируем их по частоте употребления и выведем 20 наиболее часто встречающихся.*

In [11]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:20]

[('не', 69472),
 ('и', 55166),
 ('в', 52902),
 ('я', 52818),
 ('RT', 38070),
 ('на', 35759),
 ('http', 32998),
 ('что', 31541),
 ('с', 27217),
 ('а', 26860),
 ('...', 22363),
 ('меня', 20656),
 ('у', 18928),
 ('как', 18279),
 ('так', 16839),
 ('D', 16575),
 ('это', 16542),
 ('мне', 16337),
 ('все', 14763),
 ('ты', 13412)]

### Задание 1.

*Сначала создадим токены с высокой, средней и низкой частотой.*

*В качестве высокочастотных определим токены, которые встречаются более 2000 раз, среднечастотные - встречаются от 500 раз до 2000 раз, остальные будем считать низкочастотными.*

*Разделение на частоты является условным и носит экспериментальный характер.*

In [12]:
high_tokens = set()
medium_tokens = set()
low_tokens = set()

for i in freq_dict_sorted:
    if i[1] > 2000:
        high_tokens.add(i[0])
    elif i[1] < 500:
        low_tokens.add(i[0])
    else:
        medium_tokens.add(i[0])

print(f'high-tokens - {len(high_tokens)}')
print(f'medium-tokens - {len(medium_tokens)}')
print(f'low-tokens - {len(low_tokens)}')

high-tokens - 132
medium-tokens - 404
low-tokens - 350587


*Возьмем стандартную модель, обучим ее, выполним предсказание на наших токенах и выведем результат.*

In [13]:
def Token_Classificator (stop_words_list):

    vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words_list)
    bow = vec.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))

    print(classification_report(pred, y_test))

*Высокочастотные токены.*

In [14]:
Token_Classificator(noise + list(low_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.77      0.60      0.67     35926
    positive       0.50      0.69      0.58     20783

    accuracy                           0.63     56709
   macro avg       0.63      0.64      0.63     56709
weighted avg       0.67      0.63      0.64     56709



*Среднечастотные токены*

In [15]:
Token_Classificator(noise + list(high_tokens) + list(low_tokens))

              precision    recall  f1-score   support

    negative       0.51      0.67      0.58     21291
    positive       0.76      0.62      0.68     35418

    accuracy                           0.64     56709
   macro avg       0.64      0.64      0.63     56709
weighted avg       0.67      0.64      0.64     56709



*Низкочастотные токены*

In [16]:
Token_Classificator(noise + list(high_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.79      0.73      0.76     30544
    positive       0.71      0.78      0.74     26165

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



#### ВЫВОД: наилучшая точность предсказания у модели с низкочастотными токенами. Однако, стоит заметить, что на точность так же оказывает влияние и количество токенов (количество низкочастотных самое высокое). Если смещать границы и увеличивать количество токенов в группе - то точность тоже начнет увеличиваться. 

### Задание 2.

Попробуем поработать с пунктуацией. Наше предположение построим на том, что в некоторых случаях пунктуация помогает в классификациях текстов. Посмотрим - просматривается ли какая зависимость в нашем случае.

In [17]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27901
    positive       1.00      1.00      1.00     28808

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [18]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
cool_token = '@'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.57      0.56      0.57     28370
    positive       0.57      0.58      0.57     28339

    accuracy                           0.57     56709
   macro avg       0.57      0.57      0.57     56709
weighted avg       0.57      0.57      0.57     56709



In [20]:
cool_token = '&'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.99      0.49      0.66     56308
    positive       0.01      0.50      0.01       401

    accuracy                           0.49     56709
   macro avg       0.50      0.50      0.34     56709
weighted avg       0.99      0.49      0.65     56709



In [21]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32812
    positive       0.83      1.00      0.91     23897

    accuracy                           0.91     56709
   macro avg       0.92      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



In [22]:
tokens_arr = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    tokens_arr.append((i,accuracy_score(pred, y_test)))
print(tokens_arr)

[('!', 0.5173781939374702), ('"', 0.5057750974272162), ('#', 0.5045230915727662), ('$', 0.49272602232449875), ('%', 0.4948244546720979), ('&', 0.4927965578655945), ("'", 0.4927436562097727), ('(', 0.02521645594173764), (')', 0.9141406126011744), ('*', 0.5122643672080269), ('+', 0.4933079405385389), (',', 0.5026362658484544), ('-', 0.5099895960076883), ('.', 0.5056692941155725), ('/', 0.54622723024564), (':', 0.5491368213158405), (';', 0.4968347175933273), ('<', 0.4927436562097727), ('=', 0.49196776525771924), ('>', 0.4927436562097727), ('?', 0.5035355939974254), ('@', 0.5699624398243665), ('[', 0.49263785289812906), ('\\', 0.49276129009504666), (']', 0.4927965578655945), ('^', 0.49955033592551445), ('_', 0.5193002874323299), ('`', 0.49233807684847203), ('{', 0.49272602232449875), ('|', 0.4887936659084096), ('}', 0.49272602232449875), ('~', 0.4926907545539509)]


**В общем-то получили достаточно предсказуемый результат: само собой разумеется, что символ ")" чаще всего будет встречаться в позитивных твитах. А слеовательно, знаки пунктуации в твитах могут быть полезными.**

### Задание 3.

Построим модели, используя другие векторайзеры и сравним результаты.

**CountVectorizer**

In [40]:
count_vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)
bow = count_vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(count_vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.80      0.77      0.78     29244
    positive       0.76      0.80      0.78     27465

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



**TfidfVectorizer**

In [41]:
tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.77      0.78      0.77     27771
    positive       0.78      0.78      0.78     28938

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



**HashingVectorizer**

In [25]:
from sklearn.feature_extraction.text import HashingVectorizer

Одновременно попробуем подобрать наилучший размер HashingVectorizer

In [26]:
for n in range (4, 24, 2):
    vectorizer = HashingVectorizer(n_features=2**n,)
    bow = vectorizer.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vectorizer.transform(x_test))
    print(f'n_features = {2**n}')
    print(classification_report(pred, y_test))
    print(56*'=')

n_features = 16
              precision    recall  f1-score   support

    negative       0.50      0.54      0.52     26047
    positive       0.58      0.55      0.56     30662

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.54     56709

n_features = 64
              precision    recall  f1-score   support

    negative       0.55      0.57      0.56     26806
    positive       0.60      0.58      0.59     29903

    accuracy                           0.58     56709
   macro avg       0.58      0.58      0.58     56709
weighted avg       0.58      0.58      0.58     56709

n_features = 256
              precision    recall  f1-score   support

    negative       0.60      0.61      0.60     27294
    positive       0.63      0.62      0.62     29415

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61  

Оптимальное количество фичей в модели с HashingVectorizer - 262144. Так как при дальнейшем увеличении размера векторайзера показатели модели существенно не улучшаются, но сильно возрастает время вычисления.

**Напишем и обучим нейронную сеть**

In [27]:
!pip install tensorflow

In [28]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

2022-10-17 19:36:50.050098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
y_train_bin = np.where(y_train=='positive', 1, 0)
y_test_bin = np.where(y_test=='positive', 1, 0)

In [30]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train_bin))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test_bin))

2022-10-17 19:37:00.051551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
train_data = train_data.batch(256)
valid_data = valid_data.batch(256)

In [32]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [33]:
def custom_standardization(input_data):
    
    return input_data

vocab_size = 15000
seq_len = 140

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [34]:
embedding_dim=250

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(1)
])

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
model.fit(train_data, validation_data=valid_data, epochs=2)

Epoch 1/2
665/665 [==============================] - 2857s 4s/step - loss: 0.4968 - accuracy: 0.6915 - val_loss: 0.4128 - val_accuracy: 0.7241
Epoch 2/2
665/665 [==============================] - 3509s 5s/step - loss: 0.4434 - accuracy: 0.7387 - val_loss: 0.5387 - val_accuracy: 0.7198


In [37]:
logits = model.predict(valid_data)
prediction = tf.round(tf.nn.sigmoid(logits))
class_preds = np.where(prediction==1, 'positive', 'negative')

222/222 [==============================] - 286s 1s/step


Построим матрицу ошибок

In [38]:
print(classification_report(class_preds, y_test))

              precision    recall  f1-score   support

    negative       0.48      0.85      0.62     15943
    positive       0.92      0.65      0.76     40766

    accuracy                           0.70     56709
   macro avg       0.70      0.75      0.69     56709
weighted avg       0.79      0.70      0.72     56709



**Сеть в этой конфигурациии на этом датасете показала не самый лучший результат и не смогла обогнать классические модели.**